In [4]:
import cv2
import numpy as np
import torch
import pandas as pd


def draw_labels_on_image(image_path, labels_file):
    """
    Loads an image from image_path, retrieves ground truth (x1, y1, x2, y2) from labels.txt,
    and displays the image with red circles at those coordinates.

    Args:
        image_path (str): Path to the image.
        labels_file (str): Path to the labels file (CSV or TXT with x1, y1, x2, y2).
    """
    # Load the image using OpenCV
    image = cv2.imread(image_path)
    if image is None:
        print(f"Unable to load image: {image_path}")
        return

    # Resize the image to (width=340, height=410) to match the model's expected size
    image_resized = image#cv2.resize(image, (340, 410))

    # Read labels from the file
    labels_df = pd.read_csv(labels_file)  # Ensure labels.txt is formatted correctly
    image_name = image_path.split('/')[-1]  # Extract filename from path

    # Find the row corresponding to the image name (assuming there is an 'id' or filename column)
    if 'img_name' in labels_df.columns:
        row = labels_df[labels_df['img_name'] == image_name]
    else:
        row = labels_df.iloc[0]  # If there's no ID column, just use the first row (for testing)

    if row.empty:
        print(f"No labels found for {image_name}")
        return

    # Extract ground truth coordinates
    x1, y1, x2, y2 = row[['x1', 'y1', 'x2', 'y2']].to_numpy().flatten()
    x1 = int(x1)
    x2 = int(x2)
    y1 = int(y1)
    y2 = int(y2)
    print(f"Ground truth coordinates: x1={x1}, y1={y1}, x2={x2}, y2={y2}")

    # Draw red circles at the ground truth coordinates
    image_drawn = image_resized.copy()
    cv2.circle(image_drawn, (x1, y1), radius=5, color=(0, 255, 0), thickness=-1)  # Green circle
    cv2.circle(image_drawn, (x2, y2), radius=5, color=(0, 0, 255), thickness=-1)  # Red circle

    # Display the image with the drawn points
    cv2.imshow("img",image_drawn)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

extract_dir = 'extracted_captchas'
processed_images_dir = "processed_captchas"
image_path = f"{extract_dir}/captchas_saved/captcha_8.png"
labels_file = "labels.txt"
draw_labels_on_image(image_path,labels_file)
image_path = f"truncated_captchas/captcha_8.png"
labels_file = "truncated_labels.csv"
draw_labels_on_image(image_path,labels_file)


Ground truth coordinates: x1=105, y1=175, x2=180, y2=207
Ground truth coordinates: x1=40, y1=75, x2=115, y2=107


In [ ]:
import cv2
import numpy as np

def show_pixels(image_path,top_pixels,bot_pixels,left_pixels,right_pixels):
    """
    Displays the top 20 rows of an image.

    Args:
        image_path (str): Path to the image file.
    """
    # Load the image
    image = cv2.imread(image_path)

    # Check if the image was loaded correctly
    if image is None:
        print(f"Error: Unable to load image at {image_path}")
        return

    truncated_image = image[bot_pixels:top_pixels, left_pixels:right_pixels]

    # Display the cropped section
    cv2.imshow("img",truncated_image)  # Use cv2_imshow in Google Colab, replace with cv2.imshow() for local use
    cv2.waitKey(0)
    cv2.destroyAllWindows()


extract_dir = 'extracted_captchas'
processed_images_dir = "processed_captchas"
image_path = f"{extract_dir}/captchas_saved/captcha_315.png"
labels_file = "labels.txt"
top_pixels,bot_pixels,left_pixels,right_pixels = 310,100,65,275 # Field coordonates
# top_pixels,bot_pixels,left_pixels,right_pixels = 55,5,185,235 # Draw 1
# top_pixels, bot_pixels, left_pixels, right_pixels = 50,10, 265, 305 #Draw 2
show_pixels(image_path,top_pixels,bot_pixels,left_pixels,right_pixels)

In [ ]:
import pandas as pd

def adjust_x_coordinates(label, left_pixels):
    return label - left_pixels

def adjust_y_coordinates(label, bot_pixels):
    return label - bot_pixels

def truncate_labels(df, left_pixels = 65, bot_pixels = 100):
    df[['x1', 'y1', 'x2', 'y2']] = df[['x1', 'y1', 'x2', 'y2']].apply(pd.to_numeric)
    df['x1'] = df['x1'].apply(lambda x: adjust_x_coordinates(x, left_pixels))
    df['x2'] = df['x2'].apply(lambda x: adjust_x_coordinates(x, left_pixels))
    df['y1'] = df['y1'].apply(lambda y: adjust_y_coordinates(y, bot_pixels))
    df['y2'] = df['y2'].apply(lambda y: adjust_y_coordinates(y, bot_pixels))
    return df

# top_pixels,bot_pixels,left_pixels,right_pixels = 310,100,65,275 # Field coordonates
# Load the labels file
df = pd.read_csv("labels.txt")
df.pipe(truncate_labels)
# Save the adjusted labels
df.to_csv('truncated_labels.csv', index=False)

# Display the updated DataFrame
print(df)


            img_name          x1          y1          x2          y2
0      captcha_1.png   46.816017  146.209957   54.581169   18.639610
1     captcha_10.png   54.581169  143.991342  155.528139   67.449134
2    captcha_100.png  103.390693  192.800866   22.411255  191.691558
3    captcha_101.png  142.216450  125.133117   60.127706  165.068182
4    captcha_102.png   95.625541  130.679654  155.528139  170.614719
..               ...         ...         ...         ...         ...
311   captcha_95.png   83.423160   35.279221  122.248918  182.817100
312   captcha_96.png   49.034632   40.825758   44.597403  172.833333
313   captcha_97.png   45.706710  170.614719  138.888528   92.963203
314   captcha_98.png   45.706710  105.165584  176.604978   49.700216
315   captcha_99.png   42.378788  187.254329   23.520563  117.367965

[316 rows x 5 columns]


In [ ]:
import os
import cv2

def crop_and_save_images(input_folder, output_folder="field",top_pixels= 330, bot_pixels= 55, left_pixels= 20, right_pixels = 320):
    """
    Crops images in the input_folder according to predefined pixel boundaries
    and saves them to the output_folder with the same filenames.

    Args:
        input_folder (str): Path to the folder containing the original images.
        output_folder (str): Path where the cropped images will be saved.
    """
    # Define cropping boundaries
    

    # Create output folder if it does not exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Process each image in the input folder
    for filename in os.listdir(input_folder):
        input_path = os.path.join(input_folder, filename)
        output_path = os.path.join(output_folder, filename)

        # Read the image
        image = cv2.imread(input_path)
        if image is None:
            print(f"Skipping {filename} (could not load image)")
            continue

        # Apply cropping
        cropped_image = image[bot_pixels:top_pixels, left_pixels:right_pixels]

        # Save the cropped image to the new folder
        cv2.imwrite(output_path, cropped_image)
        print(f"Saved cropped image: {output_path}")

# top_pixels,bot_pixels,left_pixels,right_pixels = 50,10,190,230 # Draw 1
# top_pixels, bot_pixels, left_pixels, right_pixels = 50,10, 265, 305 #Draw 2

# Example Usage
top_pixels,bot_pixels,left_pixels,right_pixels = 310,100,65,275 # Field coordonates
crop_and_save_images("extracted_captchas/captchas_saved","truncated_captchas",top_pixels,bot_pixels,left_pixels,right_pixels) #truncated captchas
# top_pixels,bot_pixels,left_pixels,right_pixels = 55,5,185,235
# crop_and_save_images("extracted_captchas/captchas_saved","draw1",top_pixels,bot_pixels,left_pixels,right_pixels) #Draw 1
# top_pixels, bot_pixels, left_pixels, right_pixels = 55,5, 260, 310
# crop_and_save_images("extracted_captchas/captchas_saved","draw2",top_pixels,bot_pixels,left_pixels,right_pixels) #Draw 2


Saved cropped image: truncated_captchas\captcha_1.png
Saved cropped image: truncated_captchas\captcha_10.png
Saved cropped image: truncated_captchas\captcha_100.png
Saved cropped image: truncated_captchas\captcha_101.png
Saved cropped image: truncated_captchas\captcha_102.png
Saved cropped image: truncated_captchas\captcha_103.png
Saved cropped image: truncated_captchas\captcha_104.png
Saved cropped image: truncated_captchas\captcha_105.png
Saved cropped image: truncated_captchas\captcha_106.png
Saved cropped image: truncated_captchas\captcha_107.png
Saved cropped image: truncated_captchas\captcha_108.png
Saved cropped image: truncated_captchas\captcha_109.png
Saved cropped image: truncated_captchas\captcha_11.png
Saved cropped image: truncated_captchas\captcha_110.png
Saved cropped image: truncated_captchas\captcha_111.png
Saved cropped image: truncated_captchas\captcha_112.png
Saved cropped image: truncated_captchas\captcha_113.png
Saved cropped image: truncated_captchas\captcha_114.

In [10]:
import cv2
import numpy as np
import os


def make_mean_image(image_folder, output_name = "mean_image.png"):
    # List all image files in the folder
    image_files = [f for f in os.listdir(image_folder) if f.endswith('.png')]  # Change extension if needed

    # Initialize an accumulator with zeros (assuming images are the same size)
    num_images = len(image_files)
    if num_images == 0:
        raise ValueError("No images found in the specified folder.")

    # Load the first image to get dimensions
    first_image = cv2.imread(os.path.join(image_folder, image_files[0]), cv2.IMREAD_COLOR)
    h, w, c = first_image.shape
    mean_image = np.zeros((h, w, c), dtype=np.float32)

    # Compute the sum of all images
    for file in image_files:
        img = cv2.imread(os.path.join(image_folder, file), cv2.IMREAD_COLOR)
        if img is None:
            print(f"Warning: Could not read {file}")
            continue
        mean_image += img.astype(np.float32)  # Convert to float to prevent overflow

    # Compute the mean by dividing by the number of images
    mean_image /= num_images

    # Convert back to uint8 format for visualization and saving
    mean_image = np.clip(mean_image, 0, 255).astype(np.uint8)

    # Save the mean image
    cv2.imwrite(output_name, mean_image)

    # Display the mean image (optional)
    cv2.imshow("Mean Image", mean_image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# Path to the folder containing the images
image_folder = "extracted_captchas/captchas_saved"  # Change this to your actual folder path
make_mean_image(image_folder)


In [ ]:
import cv2
import numpy as np
import os

def retrieve_mean_image(image_folder = "truncated_captchas", output_folder = "filtered_truncated_captchas", mean_image_path = "mean_image.png"):
    # Create output folder if it doesn't exist
    os.makedirs(output_folder, exist_ok=True)

    # Load the mean image
    mean_image = cv2.imread(mean_image_path, cv2.IMREAD_COLOR).astype(np.float32)

    # List all image files in the folder
    image_files = [f for f in os.listdir(image_folder) if f.endswith('.png')]  # Change extension if needed

    # Process each image
    for file in image_files:
        img_path = os.path.join(image_folder, file)
        output_path = os.path.join(output_folder, file)
        
        # Load the image
        img = cv2.imread(img_path, cv2.IMREAD_COLOR).astype(np.float32)

        # Subtract the mean image
        cleaned_img = img - mean_image

        # Normalize back to 0-255
        cleaned_img = np.clip(cleaned_img, 0, 255).astype(np.uint8)

        # Save the cleaned image
        cv2.imwrite(output_path, cleaned_img)

    print(f"Cleaned images saved in {output_folder}")

# Paths
image_folder = "truncated_captchas"  # Change this to your actual folder
output_folder = "filtered_truncated_captchas"  # Folder to save cleaned images
mean_image_path = "mean_image.png"  # Path to saved mean image


Cleaned images saved in filtered_truncated_captchas


In [11]:
import cv2
import numpy as np
import torch
import pandas as pd


def draw_labels_on_image(image_path, labels_file):
    """
    Loads an image from image_path, retrieves ground truth (x1, y1, x2, y2) from labels.txt,
    and displays the image with red circles at those coordinates.

    Args:
        image_path (str): Path to the image.
        labels_file (str): Path to the labels file (CSV or TXT with x1, y1, x2, y2).
    """
    # Load the image using OpenCV
    image = cv2.imread(image_path)
    if image is None:
        print(f"Unable to load image: {image_path}")
        return

    # Resize the image to (width=340, height=410) to match the model's expected size
    image_resized = image#cv2.resize(image, (340, 410))

    # Read labels from the file
    labels_df = pd.read_csv(labels_file)  # Ensure labels.txt is formatted correctly
    image_name = image_path.split('/')[-1]  # Extract filename from path

    # Find the row corresponding to the image name (assuming there is an 'id' or filename column)
    if 'img_name' in labels_df.columns:
        row = labels_df[labels_df['img_name'] == image_name]
    else:
        row = labels_df.iloc[0]  # If there's no ID column, just use the first row (for testing)

    if row.empty:
        print(f"No labels found for {image_name}")
        return

    # Extract ground truth coordinates
    x1, y1, x2, y2 = row[['x1', 'y1', 'x2', 'y2']].to_numpy().flatten()
    x1 = int(x1)
    x2 = int(x2)
    y1 = int(y1)
    y2 = int(y2)
    print(f"Ground truth coordinates: x1={x1}, y1={y1}, x2={x2}, y2={y2}")

    # Draw red circles at the ground truth coordinates
    image_drawn = image_resized.copy()
    cv2.circle(image_drawn, (x1, y1), radius=5, color=(0, 255, 0), thickness=-1)  # Green circle
    cv2.circle(image_drawn, (x2, y2), radius=5, color=(0, 0, 255), thickness=-1)  # Red circle

    # Display the image with the drawn points
    cv2.imshow("img",image_drawn)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

image_path = f"filtered_truncated_captchas/captcha_8.png"
labels_file = "truncated_labels.csv"
draw_labels_on_image(image_path,labels_file)

Ground truth coordinates: x1=40, y1=75, x2=115, y2=107


In [1]:
import os
import cv2
import numpy as np
from tqdm import tqdm  # Progress bar

def compute_mean_std(input_folder):
    """Compute the mean and std for each RGB channel across all images."""
    image_list = []
    
    for filename in tqdm(os.listdir(input_folder), desc="Computing mean & std"):
        img_path = os.path.join(input_folder, filename)
        img = cv2.imread(img_path)  # Read image in BGR format
        if img is not None:
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert to RGB
            image_list.append(img)

    # Convert list to a big numpy array (N, H, W, C)
    image_array = np.stack(image_list, axis=0).astype(np.float32) 

    # Compute mean and std along (N, H, W) axis → (C,)
    mean = np.mean(image_array, axis=(0, 1, 2))
    std = np.std(image_array, axis=(0, 1, 2))
    print(f"moyenne = {mean} \n std={std}")
    return mean, std

def normalize_images(input_folder, output_folder):
    """Normalize RGB channels of all images in input_folder and save them to output_folder."""
    os.makedirs(output_folder, exist_ok=True)  # Create output folder if not exists

    # Compute mean and std
    mean, std = compute_mean_std(input_folder)
    print(mean,std)
    for filename in tqdm(os.listdir(input_folder), desc="Normalizing images"):
        img_path = os.path.join(input_folder, filename)
        img = cv2.imread(img_path)  # Read image in BGR format
        if img is None:
            print(f"Skipping {filename} (unable to read)")
            continue

        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert to RGB
        img = img.astype(np.float32) 

        # Normalize: (pixel - mean) / std
        img = (img - mean) / std

        # Convert back to 0-255 range for saving
        img = ((img - img.min()) / (img.max() - img.min()) * 255).astype(np.uint8)

        # Save processed image
        output_path = os.path.join(output_folder, filename)
        cv2.imwrite(output_path, cv2.cvtColor(img, cv2.COLOR_RGB2BGR))  # Convert back to BGR

    print(f"✅ Normalized images saved in: {output_folder}")


# Example Usage
input_folder = "truncated_captchas"
output_folder = "normalized_images"
normalize_images(input_folder, output_folder)


Computing mean & std: 100%|██████████| 316/316 [00:00<00:00, 1707.75it/s]


moyenne = [214.72969 217.93405 141.7747 ] 
 std=[51.95303  47.542946 46.740856]
[214.72969 217.93405 141.7747 ] [51.95303  47.542946 46.740856]


Normalizing images: 100%|██████████| 316/316 [00:00<00:00, 354.55it/s]

✅ Normalized images saved in: normalized_images


In [ ]:
import os
import cv2
import numpy as np
from tqdm import tqdm
import pandas as pd

def apply_existing_normalization(input_folder, output_folder, mean, std):
    """
    Applies the same mean and std normalization (computed earlier) to new images.

    Args:
        input_folder (str): Path to the folder containing new images.
        output_folder (str): Path to save the normalized images.
        mean (tuple): Mean values of the original dataset (R, G, B).
        std (tuple): Standard deviation values of the original dataset (R, G, B).
    """
    os.makedirs(output_folder, exist_ok=True)  # Create the output folder if it doesn't exist
    df = pd.DataFrame([])
    for filename in tqdm(os.listdir(input_folder), desc="Processing new images"):
        img_path = os.path.join(input_folder, filename)
        img = cv2.imread(img_path)  # Read image in BGR format
        if img is None:
            print(f"Skipping {filename} (unable to read)")
            continue
        
        # Convert to RGB and scale pixel values to [0,1]
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB).astype(np.float32) / 255.0

        # Apply normalization using precomputed mean & std
        img_normalized = (img - mean) / std  # Standardize the image

        # Rescale to 0-255 for saving
        img_normalized = ((img_normalized - img_normalized.min()) / (img_normalized.max() - img_normalized.min()) * 255).astype(np.uint8)
        df = pd.concat([df, pd.DataFrame({"img_name": [filename], "min": [img_normalized.min()], "max": [img_normalized.max()]})], ignore_index=True)

        # Convert back to BGR before saving (OpenCV expects BGR format)
        output_path = os.path.join(output_folder, filename)
        cv2.imwrite(output_path, cv2.cvtColor(img_normalized, cv2.COLOR_RGB2BGR))
    df.to_csv(input_folder+".csv")
    print(f"✅ Normalized images saved in: {output_folder}")

# Valeurs trouvées:
mean = (0.83977205, 0.8524061, 0.55467314)  # Dataset's mean
std = (0.2027646, 0.18541439, 0.18301369)  # Dataset's std

input_folder = "premier_perso"
output_folder = "normalized_premier_perso"
apply_existing_normalization(input_folder, output_folder, mean, std)

input_folder = "second_perso"
output_folder = "normalized_second_perso"
apply_existing_normalization(input_folder, output_folder, mean, std)


Processing new images: 100%|██████████| 316/316 [00:00<00:00, 880.68it/s]


✅ Normalized images saved in: normalized_premier_perso


Processing new images: 100%|██████████| 316/316 [00:00<00:00, 889.32it/s]

✅ Normalized images saved in: normalized_second_perso


In [ ]:
import numpy as np
def encode_image(img, original_min, original_max):
    """
    Reverts the min-max scaling applied to an image.

    Args:
        img (numpy.ndarray): The encoded image (after min-max scaling to 255).
        original_min (float): The minimum pixel value before encoding.
        original_max (float): The maximum pixel value before encoding.

    Returns:
        numpy.ndarray: The encoded image with values standardized
    """
    img = img.astype(np.float32)  # Convert to float32 for precision

    # Reverse min-max scaling: 
    img_encoded = img / 255.0 * (original_max - original_min) + original_min

    return img_encoded


In [53]:
extract_dirs = ['normalized_images',"normalized_premier_perso","normalized_second_perso"]
for extract_dir in extract_dirs:
    image_path = f"{extract_dir}/captcha_8.png"
    image = cv2.imread(image_path)
    print(image.shape)

(210, 210, 3)
(50, 50, 3)
(50, 50, 3)


In [ ]:
import pandas as pd 
import random

def augmentation(df: pd.DataFrame) -> pd.DataFrame:
    """
    Create augmented dataframe by adding the name of the rotated images and noise to the coordonates
    """

    augmented_data = []  # List to collect all rows for the new dataframe
    for _, row in df.iterrows():  # iterrows returns index and row (row is a pandas Series)
        for i in range(1, 5):  # Iterate through 1 to 4 for the rotations
            # Create the new img_name for each rotated image
           
            img_name = f'{row["img_name"][:-4]}_rotated_{i}.png'
            # Create a new row with the same coordinates but updated image name
            new_row = {
                df.columns[0]: img_name,
                df.columns[1]: (row[df.columns[1]] + 3*(1/2-random.random()))/210,
                df.columns[2]: (row[df.columns[2]] + 3*(1/2-random.random()))/210,
                df.columns[3]: (row[df.columns[3]] + 3*(1/2-random.random()))/210,
                df.columns[4]: (row[df.columns[4]] + 3*(1/2-random.random()))/210
            }
            
            # Append the new row to the augmented_data list
            augmented_data.append(new_row)

    # Convert the list of new rows to a DataFrame
    augmented_df = pd.DataFrame(augmented_data)
    
    return augmented_df


# Load the original dataframe
df = pd.read_csv("truncated_labels.csv")

# Apply augmentation to the dataframe
augmented_df = augmentation(df)

# Print the result (or save it to a CSV file)
print(augmented_df)
# Optionally, save the new augmented dataframe to a CSV
augmented_df.to_csv("training/augmented_labels.csv", index=False)


                      img_name        x1        y1        x2        y2
0      captcha_1_rotated_1.png  0.227244  0.689850  0.264742  0.092760
1      captcha_1_rotated_2.png  0.229355  0.702371  0.256679  0.090055
2      captcha_1_rotated_3.png  0.223377  0.691245  0.257841  0.085348
3      captcha_1_rotated_4.png  0.221414  0.695206  0.252786  0.089547
4     captcha_10_rotated_1.png  0.253058  0.687196  0.746676  0.322271
...                        ...       ...       ...       ...       ...
1259  captcha_98_rotated_4.png  0.224242  0.494220  0.846591  0.235562
1260  captcha_99_rotated_1.png  0.202343  0.896912  0.118106  0.559476
1261  captcha_99_rotated_2.png  0.203476  0.895013  0.116340  0.558038
1262  captcha_99_rotated_3.png  0.206369  0.898739  0.117417  0.554189
1263  captcha_99_rotated_4.png  0.201659  0.888489  0.112035  0.563014

[1264 rows x 5 columns]


In [12]:
from PIL import Image
import os
#Creation of augmented data for dataset
def rotate_image(image_path, num_rotations=4, output_dir="augmented_images", rotate = True):
    """
    Rotates an image 90 degrees num_rotations times and saves the result.

    Args:
        image_path (str): Path to the original image.
        num_rotations (int): Number of 90 degree rotations to apply.
        output_dir (str): Directory to save the rotated images. Default is "augmented_images".
    
    Returns:
        list: A list of file paths to the saved rotated images.
    """
    # Load the image
    img = Image.open(image_path)
    # Prepare the output directory
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    # Rotate the image the specified number of times
    for i in range(num_rotations):
        if rotate:
            img = img.rotate(90, expand=True)  # Rotate 90 degrees and expand the image size if necessary
        # Create a new file name for the rotated image
        for j in range(1,4):
            rotated_image_path = os.path.join(output_dir, f"{os.path.splitext(os.path.basename(image_path))[0]}_rotated_{i+1}.png")
        # Save the rotated image
        img.save(rotated_image_path)

    
def gen_datasets(norm=True):
    if norm : 
        norm = "normalized_"
    else : 
        norm = ""
    image_folder = norm + "truncated_captchas/"
    count = 0
    for filename in os.listdir(image_folder):
        if count > 252 : break
        count+=1
        rotate_image(image_folder+filename,output_dir="training/train_set/field/",rotate=False)
    # image_folder = "normalized_second_perso/"
    count = 0 
    for filename in os.listdir(image_folder):
        if count <253: 
            count+=1
            continue
        rotate_image(image_folder+filename,output_dir="training/test_set/field/",rotate=False)

    image_folder = norm + "premier_perso/"
    count = 0
    for filename in os.listdir(image_folder):
        if count > 252 : break
        count+=1
        rotate_image(image_folder+filename,output_dir="training/train_set/rotated_draw1/")
    # image_folder = "normalized_second_perso/"
    count = 0 
    for filename in os.listdir(image_folder):
        if count <253: 
            count+=1
            continue
        rotate_image(image_folder+filename,output_dir="training/test_set/rotated_draw1/")

    image_folder = norm + "second_perso/"
    count = 0
    for filename in os.listdir(image_folder):
        if count > 252 : break
        count+=1
        rotate_image(image_folder+filename,output_dir="training/train_set/rotated_draw2/")
    # image_folder = "normalized_second_perso/"
    count = 0 
    for filename in os.listdir(image_folder):
        if count <253: 
            count+=1
            continue
        rotate_image(image_folder+filename,output_dir="training/test_set/rotated_draw2/")
    print("Done !")



gen_datasets(norm=False)
    

Done !


In [11]:
import pandas as pd 

def save_csv_for_training(df,draw=1,train_test_split=0.8):
    """
    Saves the csv containing the labels in the appropriate folder for the training and evaluating datasets

    Args:
        df (pd.Dataframe): Dataframe containing the augmented dataframe containing this columns ["img_name", "x1", "y1", "x2", "y2"]
        draw (int): 1 or 2 to chose the coordonate to remove
        train_test_split (float) : The proportion of images for the training
    """
    L = [1,2]
    df = df.drop([f"x{L[-draw]}",f"y{L[-draw]}"],axis=1)
    df.columns = ["img_name","x","y"]
    treshold = (int(train_test_split*len(df)/4)+1)*4 #Treshold adapted to the train_test_split
    df[treshold:].to_csv(f"training/test_set/rotated_draw{draw}/augmented_labels.csv")
    df[:treshold].to_csv(f"training/train_set/rotated_draw{draw}/augmented_labels.csv")

# Load the original dataframe
df = pd.read_csv("training/augmented_labels.csv")
save_csv_for_training(df,draw=1)
save_csv_for_training(df,draw=2)


In [4]:
[1,2][-1]

2

In [5]:
import numpy as np
np.sqrt(0.0066)*210*210/2

np.float64(1791.3504682222292)